In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/physionet-ecg/scg_rhc_f16.parquet


In [4]:
# ===============================
# Install dependencies (run once)
# ===============================
# (Uncomment if you need to install)
# !pip install --quiet neurokit2 xgboost joblib scikit-learn scipy tqdm

# ===============================
# Full, robust HRV -> XGBoost pipeline
# ===============================
import os
import math
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import neurokit2 as nk
from scipy.signal import resample, welch, butter, filtfilt, iirnotch

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupShuffleSplit, GroupKFold
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import silhouette_score

import xgboost as xgb
import joblib

# ---------- reproducibility ----------
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# ---------- paths & params ----------
PQ_PATH = "/kaggle/input/physionet-ecg/scg_rhc_f16.parquet"
OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)

MAX_RECS = 374         # set how many records to use (use 374 to use all)
WIN_S = 30             # window length in seconds
HOP_S = 15             # hop in seconds
CANDIDATE_FS = [125, 250, 360, 500]
TARGET_FS = 250        # not strictly required for XGB, but useful if you also want fixed-length windows

print("Loading parquet:", PQ_PATH)
df = pd.read_parquet(PQ_PATH)
print("Loaded parquet:", PQ_PATH, "shape:", df.shape)

# choose ECG column
preferred = ["ECG_lead_II", "ECG_lead_ii", "ECG_lead_II ", "ECG_lead_I", "ecg_lead_ii"]
ecg_col = None
for p in preferred:
    if p in df.columns:
        ecg_col = p
        break
if ecg_col is None:
    for c in df.columns:
        if "ecg" in c.lower():
            ecg_col = c
            break
if ecg_col is None:
    raise RuntimeError("No ECG column found in parquet.")
print("Using ECG column:", ecg_col)

# ---------------------------
# helper: filters & fs estimation
# ---------------------------
def bandpass(sig, fs, low=0.5, high=40, order=4):
    b, a = butter(order, [low/(fs/2), high/(fs/2)], btype='band')
    return filtfilt(b, a, sig)

def notch(sig, fs, freq=50.0, q=30.0):
    b, a = iirnotch(freq/(fs/2), q)
    return filtfilt(b, a, sig)

def estimate_fs_for_signal(arr, candidates=CANDIDATE_FS):
    arr = np.asarray(arr, dtype=float)
    best_fs = None
    best_score = 1e9
    for fs in candidates:
        try:
            peaks_info = nk.ecg_peaks(arr, sampling_rate=fs)[1]
            peaks = peaks_info.get("ECG_R_Peaks", [])
            if len(peaks) < 3:
                continue
            rr = np.diff(peaks) / fs
            mean_hr = 60.0 / rr.mean()
            score = 0 if (40 <= mean_hr <= 140) else abs(mean_hr - 80)
            if score < best_score:
                best_score = score
                best_fs = fs
        except Exception:
            continue
    return best_fs

# ---------------------------
# robust HRV feature extraction per window
# returns dict or None
# ---------------------------
def extract_hrv_features_from_window(w, fs):
    w = np.asarray(w, dtype=float)
    if w.size == 0 or np.max(np.abs(w)) < 1e-6:
        return None
    # try clean/filter
    try:
        wf = bandpass(w, fs)
        wf = notch(wf, fs, freq=50.0)
    except Exception:
        wf = w.copy()
    # R-peak detection
    try:
        signals, info = nk.ecg_process(wf, sampling_rate=fs)
        rpeaks = np.array(info.get("ECG_R_Peaks", []))
    except Exception:
        # fallback
        rpeaks = np.array(nk.ecg_peaks(wf, sampling_rate=fs)[1].get("ECG_R_Peaks", []))
    if len(rpeaks) < 4:
        return None
    rr_s = np.diff(rpeaks) / fs            # in seconds
    if rr_s.size < 2:
        return None
    rr_ms = rr_s * 1000.0                  # ms

    out = {}
    out['mean_hr'] = float(60.0 / rr_s.mean())
    out['sdnn'] = float(np.std(rr_ms, ddof=1))
    # RMSSD & pNN50
    if rr_ms.size >= 2:
        diff_rr = np.diff(rr_ms)
        out['rmssd'] = float(np.sqrt(np.mean(diff_rr**2)))
        out['pnn50'] = float(np.mean(np.abs(diff_rr) > 50.0))
    else:
        out['rmssd'] = np.nan
        out['pnn50'] = np.nan
    # Poincare SD1 / SD2
    if rr_ms.size >= 2:
        sd1 = np.sqrt(0.5) * np.std(np.diff(rr_ms))
        sd1sq = sd1**2
        sdnn = out['sdnn']
        sd2sq = max(0.0, 2.0 * (sdnn**2) - sd1sq)
        sd2 = math.sqrt(sd2sq)
        out['sd1'] = float(sd1)
        out['sd2'] = float(sd2)
    else:
        out['sd1'] = np.nan
        out['sd2'] = np.nan

    # Frequency domain via resampled tachogram -> Welch
    try:
        # times corresponding to each rr interval (use time at the end of each RR)
        times = rpeaks[1:] / float(fs)
        if len(times) >= 4:
            fs_interp = 4.0
            # create interp grid (ensure there is at least a few points)
            t0, t1 = times[0], times[-1]
            if t1 - t0 < 1.0:
                # fallback to small uniform grid
                t_interp = np.linspace(t0, t1, max(4, int((t1 - t0) * fs_interp)))
            else:
                t_interp = np.arange(t0, t1, 1.0 / fs_interp)
                if len(t_interp) < 4:
                    t_interp = np.linspace(t0, t1, max(4, int((t1 - t0) * fs_interp)))
            rr_interp = np.interp(t_interp, times, rr_ms)
            rr_interp = rr_interp - np.mean(rr_interp)
            nperseg = min(256, len(rr_interp))
            fxx, pxx = welch(rr_interp, fs=fs_interp, nperseg=nperseg)
            # LF: 0.04-0.15 Hz, HF: 0.15-0.4 Hz
            lf_mask = (fxx >= 0.04) & (fxx < 0.15)
            hf_mask = (fxx >= 0.15) & (fxx < 0.4)
            lf = float(np.trapz(pxx[lf_mask], fxx[lf_mask])) if lf_mask.any() else 0.0
            hf = float(np.trapz(pxx[hf_mask], fxx[hf_mask])) if hf_mask.any() else 0.0
            total_mask = (fxx >= 0.003) & (fxx < 0.4)
            total_power = float(np.trapz(pxx[total_mask], fxx[total_mask])) if total_mask.any() else (lf + hf)
            out['lf'] = lf
            out['hf'] = hf
            out['lf_hf'] = float(lf / hf) if (hf is not None and hf > 0) else np.nan
            out['total_power'] = total_power
        else:
            out['lf'] = np.nan; out['hf'] = np.nan; out['lf_hf'] = np.nan; out['total_power'] = np.nan
    except Exception:
        out['lf'] = np.nan; out['hf'] = np.nan; out['lf_hf'] = np.nan; out['total_power'] = np.nan

    return out

# ---------------------------
# 4) sliding windows + feature extraction
# ---------------------------
features = []
raw_windows = []
max_recs = min(MAX_RECS, len(df))
lengths = [len(np.asarray(a, float)) for a in df[ecg_col]]
print("ECG sample counts per row — min/median/max:",
      int(np.min(lengths)), int(np.median(lengths)), int(np.max(lengths)))

for idx in tqdm(range(max_recs), desc="Records"):
    arr = np.asarray(df[ecg_col].iloc[idx], dtype=float)
    fs = estimate_fs_for_signal(arr)
    if fs is None:
        # skip unusable record
        continue
    npts = arr.size
    win_pts = int(WIN_S * fs)
    hop_pts = int(HOP_S * fs)
    if npts < win_pts:
        continue
    for start in range(0, npts - win_pts + 1, hop_pts):
        w = arr[start:start + win_pts]
        feats = extract_hrv_features_from_window(w, fs)
        if feats is None:
            continue
        rec = {"rec_index": int(idx), "start_sample": int(start), "fs": int(fs), "n_samples": int(len(w))}
        rec.update(feats)
        features.append(rec)
        # store optionally the resampled window if you want DL later
        raw_windows.append(resample(w, int(round(len(w) * (TARGET_FS / fs)))))

feats_df = pd.DataFrame(features)
print("Extracted feature windows:", feats_df.shape)
if feats_df.shape[0] == 0:
    raise RuntimeError("No valid windows found; check ECG data and sampling rate detection.")

# ---------------------------
# 5) Feature cleanup & selected feature columns
# ---------------------------
candidate_cols = ['mean_hr', 'sdnn', 'rmssd', 'pnn50', 'sd1', 'sd2', 'lf', 'hf', 'lf_hf', 'total_power']
# keep only columns that exist
candidate_cols = [c for c in candidate_cols if c in feats_df.columns]
print("Using numeric features:", candidate_cols)

feats_df_clean = feats_df.dropna(subset=['mean_hr', 'sdnn']).reset_index(drop=True)  # ensure core features present
# fill other nans with median
for c in candidate_cols:
    if feats_df_clean[c].isna().any():
        feats_df_clean[c] = feats_df_clean[c].fillna(feats_df_clean[c].median())

raw_windows_clean = np.array(raw_windows, dtype=object)
# align raw_windows to feats_df_clean indices (we appended in lock-step so slicing is okay)
if len(raw_windows_clean) != len(feats_df):
    # if differing, try to align using valid_mask
    valid_mask = feats_df[candidate_cols].notna().all(axis=1).values
    raw_windows_clean = raw_windows_clean[valid_mask]
else:
    # apply same dropna mask used above
    mask = feats_df.index[feats_df[['mean_hr', 'sdnn']].notna().all(axis=1)]
    raw_windows_clean = raw_windows_clean[mask]

print("Windows after cleaning:", feats_df_clean.shape[0], " raw_windows:", raw_windows_clean.shape[0])

# ---------------------------
# 6) Unsupervised clustering -> pseudo labels (k=3) same approach as your baseline
# ---------------------------
X = feats_df_clean[candidate_cols].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = 3
km = xgb.sklearn = None  # avoid name collision (we will import KMeans)
from sklearn.cluster import KMeans
km = KMeans(n_clusters=k, random_state=SEED, n_init=20)
labels = km.fit_predict(X_scaled)
sil = silhouette_score(X_scaled, labels)
print("Silhouette score (k=3):", round(sil, 3))

centers_df = pd.DataFrame(scaler.inverse_transform(km.cluster_centers_), columns=candidate_cols)
print("Cluster centers:\n", centers_df)

# map cluster -> pseudo_depth by SDNN (desc -> light=0)
order = centers_df.sort_values('sdnn', ascending=False).index.tolist()
cluster_to_depth = {c: i for i, c in enumerate(order)}
feats_df_clean['cluster'] = labels
feats_df_clean['pseudo_depth'] = feats_df_clean['cluster'].map(cluster_to_depth).astype(float)

# ---------------------------
# 7) Group-aware train/test split (preserve rec_index grouping)
# ---------------------------
groups = feats_df_clean['rec_index'].values
X_mat = feats_df_clean[candidate_cols].values
y = feats_df_clean['pseudo_depth'].values

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
train_idx, test_idx = next(gss.split(X_mat, y, groups))
X_train, X_test = X_mat[train_idx], X_mat[test_idx]
y_train, y_test = y[train_idx], y[test_idx]
groups_train = groups[train_idx]

print("Windows total:", len(y), "Train windows:", len(y_train), "Test windows:", len(y_test))

# scale using scaler fitted above (X_scaled corresponds to whole dataset)
X_train_scaled = scaler.transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# ---------------------------
# 8) Light random search of XGBoost hyperparameters (on training folds only)
# ---------------------------
def cv_mean_mae_for_params(params, Xtr, ytr, groups_tr, n_splits=4):
    # use GroupKFold on training records
    n_groups = len(np.unique(groups_tr))
    n_splits = min(n_splits, n_groups) if n_groups >= 2 else 2
    gkf = GroupKFold(n_splits=n_splits)
    maes = []
    for tr_idx, val_idx in gkf.split(Xtr, ytr, groups_tr):
        model = xgb.XGBRegressor(random_state=SEED, n_jobs=1, **params)
        model.fit(Xtr[tr_idx], ytr[tr_idx])
        preds = model.predict(Xtr[val_idx])
        maes.append(mean_absolute_error(ytr[val_idx], preds))
    return np.mean(maes)

# parameter search space (small, safe)
param_choices = {
    "max_depth": [3, 4, 5, 6],
    "learning_rate": [0.01, 0.03, 0.05, 0.1],
    "n_estimators": [100, 200, 400, 800],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "reg_alpha": [0.0, 1e-3, 1e-2],
    "reg_lambda": [1.0, 1.5, 2.0]
}

# sample a set of candidate configs randomly (safe count)
n_trials = 24
best_mae = 1e9
best_params = None
tried = 0
print("Starting small random search over XGBoost parameter space ({} trials)".format(n_trials))
for _ in range(n_trials):
    params = {
        "max_depth": random.choice(param_choices["max_depth"]),
        "learning_rate": random.choice(param_choices["learning_rate"]),
        "n_estimators": random.choice(param_choices["n_estimators"]),
        "subsample": random.choice(param_choices["subsample"]),
        "colsample_bytree": random.choice(param_choices["colsample_bytree"]),
        "reg_alpha": random.choice(param_choices["reg_alpha"]),
        "reg_lambda": random.choice(param_choices["reg_lambda"]),
    }
    try:
        mae_cv = cv_mean_mae_for_params(params, X_train_scaled, y_train, groups_train, n_splits=4)
    except Exception as e:
        print("Trial failed:", e)
        continue
    tried += 1
    if mae_cv < best_mae:
        best_mae = mae_cv
        best_params = params
    if tried % 6 == 0:
        print(f"  tried {tried} / {n_trials} - current best MAE (cv) = {best_mae:.4f}")
print("Random search finished. trials run:", tried)
print("Best CV MAE on train folds:", best_mae)
print("Best params:", best_params)

# if tuning failed for some reason, fall back to reasonable defaults
if best_params is None:
    best_params = {"max_depth":5, "learning_rate":0.05, "n_estimators":400, "subsample":0.9, "colsample_bytree":0.9, "reg_alpha":0.0, "reg_lambda":1.0}
    print("Using fallback params:", best_params)

# ---------------------------
# 9) Train final XGB on train set and evaluate on test set
# ---------------------------
final_model = xgb.XGBRegressor(random_state=SEED, n_jobs=1, **best_params)
final_model.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)], verbose=False)
test_preds = final_model.predict(X_test_scaled)
test_mae = mean_absolute_error(y_test, test_preds)
test_r2  = r2_score(y_test, test_preds)
print("[Final XGB] Test MAE: {:.4f} | R2: {:.4f}".format(test_mae, test_r2))

# feature importance (by gain)
try:
    imp = final_model.get_booster().get_score(importance_type='gain')
    # map back to column names
    fi = {candidate_cols[int(k.replace('f',''))]: v for k, v in imp.items() if k.startswith('f')}
    print("Top features by XGBoost gain (partial):")
    # sort descending
    for k,v in sorted(fi.items(), key=lambda x: -x[1])[:10]:
        print(f"  {k}: {v:.4f}")
except Exception:
    pass

# save artifacts
joblib.dump(final_model, os.path.join(OUT_DIR, "xgb_final_model.pkl"))
joblib.dump(scaler, os.path.join(OUT_DIR, "hrv_scaler.pkl"))
joblib.dump({"candidate_cols": candidate_cols, "best_params": best_params}, os.path.join(OUT_DIR, "meta.pkl"))
print("Saved artifacts to", OUT_DIR)


Loading parquet: /kaggle/input/physionet-ecg/scg_rhc_f16.parquet
Loaded parquet: /kaggle/input/physionet-ecg/scg_rhc_f16.parquet shape: (374, 20)
Using ECG column: ECG_lead_II
ECG sample counts per row — min/median/max: 7500 7500 7500


Records:   0%|          | 0/374 [00:00<?, ?it/s]

Extracted feature windows: (148, 14)
Using numeric features: ['mean_hr', 'sdnn', 'rmssd', 'pnn50', 'sd1', 'sd2', 'lf', 'hf', 'lf_hf', 'total_power']
Windows after cleaning: 148  raw_windows: 148
Silhouette score (k=3): 0.418
Cluster centers:
      mean_hr         sdnn        rmssd     pnn50          sd1          sd2  \
0  35.054123  1008.298631  1348.083816  0.905750   949.956762  1054.452711   
1  28.656391  1379.848841  1733.567958  0.953869  1222.582365  1476.074905   
2  41.217083   625.702350   854.070917  0.777838   602.627973   640.384940   

              lf             hf      lf_hf   total_power  
0  152012.988432  156448.221074   1.368202  5.030918e+05  
1  562998.273475  107734.185549  10.616380  1.022205e+06  
2   58508.085515   60523.649095   1.682025  1.788265e+05  
Windows total: 148 Train windows: 118 Test windows: 30
Starting small random search over XGBoost parameter space (24 trials)
  tried 6 / 24 - current best MAE (cv) = 0.1141
  tried 12 / 24 - current best MAE 